In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

# cache_pathの定義
cache_path = r"C:\Users\100ca\Documents\PyCode\etf\data\etf_weekly_close_cache.pkl"

def load_cache():
    """キャッシュされたデータを読み込む関数"""
    with open(cache_path, "rb") as file:  # バイナリ読み込みモードでファイルを開く
        return pickle.load(file)

def calculate_sharpe_ratios():
    """各ETFのシャープレシオを計算し、それに基づいてETFをランキングする関数。週次データを使用。"""
    cache = load_cache()
    sharpe_ratios = {}

    for etf, records in cache.items():
        # recordsが空でない、かつ'Date'キーを含む辞書のリストであることを確認
        if records and all('Date' in record for record in records):
            try:
                prices_df = pd.DataFrame(records).set_index('Date')
                if 'Close' in prices_df.columns:
                    # 週次リターンを計算
                    returns = prices_df['Close'].pct_change()
                    # 年間リターンの平均と標準偏差を計算
                    mean_return = returns.mean() * 52
                    std_dev = returns.std() * np.sqrt(52)
                    # 無リスクレートを0と仮定してシャープレシオを計算
                    if std_dev != 0:  # 0除算を避ける
                        sharpe_ratio = mean_return / std_dev
                        sharpe_ratios[etf] = sharpe_ratio
            except Exception as e:
                print(f"Error processing {etf}: {e}")

    # シャープレシオが0.5以下のものをフィルタリング
    filtered_sharpe_ratios = {etf: ratio for etf, ratio in sharpe_ratios.items() if ratio > 0.5}

    # シャープレシオに基づいてETFをランキング
    sorted_sharpe_ratios = sorted(filtered_sharpe_ratios.items(), key=lambda x: x[1], reverse=True)
    return sorted_sharpe_ratios

def etf_prices():
    """シャープレシオ順にETFの株価をプロットし、それを4つのsubplotに分割して表示する関数"""
    cache = load_cache()
    etf_prices = {}

    # シャープレシオのランキングからETF名を取得
    ranked_etfs = [etf for etf, _ in calculate_sharpe_ratios()]

    for etf in ranked_etfs:
        if etf in cache:
            records = cache[etf]
            if records and all('Date' in record for record in records):
                try:
                    prices_df = pd.DataFrame(records).set_index('Date')
                    if 'Close' in prices_df.columns:
                        etf_prices[etf] = prices_df['Close']
                except Exception as e:
                    print(f"Error processing {etf}: {e}")

    # ETFの株価をCSVファイルに保存
    df_etf_prices = round(pd.DataFrame(etf_prices).dropna(thresh=90, axis=1),1)
    display(df_etf_prices)
    return df_etf_prices


    df_etf_prices.to_csv(r"C:\Users\100ca\Documents\PyCode\etf\data\chart.csv")

    # シャープレシオのランキングを出力
    sharpe_ratios_ranking = calculate_sharpe_ratios()
    for etf, sharpe_ratio in sharpe_ratios_ranking:
        print(f"{etf}: {sharpe_ratio:.2f}")

df_etf_prices = etf_prices()

,SMH,XLK,SOXX,IXN,QQQ,VGT,DXJ,TECL,MGK,QLD,...,DCSX,VDC,IXP,TAN,UDOW,VOX,XLB,BOTZ,HEDJ,EXI
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-04,50.5,69.8,179.5,26.9,171.2,189.9,49.5,11.7,119.0,20.4,...,NaN,139.6,55.1,23.2,45.9,82.6,54.7,19.0,31.4,86.8
2019-03-11,53.1,73.0,189.4,28.1,178.4,198.9,50.5,13.4,123.1,22.1,...,NaN,142.7,56.7,24.0,48.1,84.4,55.4,19.8,32.3,88.3
2019-03-18,53.4,73.3,190.3,28.2,178.6,198.7,49.9,13.5,122.9,22.2,...,NaN,142.7,56.5,23.3,46.0,84.0,54.3,19.8,31.5,87.2
2019-03-25,53.2,74.0,189.5,28.5,179.7,200.6,50.6,13.9,124.3,22.5,...,NaN,145.2,56.3,23.2,48.2,83.5,55.5,20.1,32.0,88.9
2019-04-01,56.3,75.9,200.7,29.3,184.7,205.6,52.5,14.9,127.1,23.7,...,NaN,143.9,57.6,24.8,51.0,86.1,57.8,21.2,33.0,91.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-29,191.2,202.2,596.5,71.5,429.0,505.2,97.1,75.6,276.0,82.7,...,0.9,196.2,81.2,42.3,79.2,128.7,83.1,29.4,44.0,128.7
2024-02-05,203.9,207.8,629.4,73.7,437.0,521.4,98.2,81.8,283.2,85.7,...,0.9,193.8,81.5,45.5,79.2,127.9,83.1,29.9,44.9,129.5
2024-02-12,201.8,202.6,625.1,71.9,430.6,509.0,100.8,75.4,279.1,83.0,...,0.9,194.5,81.1,46.9,78.8,127.5,85.2,30.0,45.2,131.1


In [10]:
def calculate_moving_52_week_return():
    """各ETFの52週間の移動平均リターンを計算する関数"""
    cache = load_cache()
    etf_returns = {}

    # シャープレシオのランキングからETF名を取得
    ranked_etfs = [etf for etf, _ in calculate_sharpe_ratios()]

    for etf in ranked_etfs:
        if etf in cache:
            records = cache[etf]
            if records and all('Date' in record for record in records):
                try:
                    prices_df = pd.DataFrame(records).set_index('Date')
                    if 'Close' in prices_df.columns:
                        # 52週間の移動平均リターンを計算
                        prices_df['52_Week_Return'] = prices_df['Close'].pct_change(periods=52)
                        etf_returns[etf] = prices_df['52_Week_Return']
                except Exception as e:
                    print(f"Error processing {etf}: {e}")

    # ETFリターンのDataFrameを作成
    df_returns = pd.DataFrame(etf_returns)
            
    # シャープレシオのランキングでdf_returnsを並び替え
    df_returns = df_returns[sharpe_ratios_ranking]
        
    return df_returns

def calculate_filtered_moving_52_week_return():
    """52週リターンが-0.4を下回ったことのある銘柄を除外して計算する関数"""
    df_returns = calculate_moving_52_week_return()
    # -0.4を下回ったことのある銘柄を除外
    filtered_etfs = df_returns.columns[(df_returns.min() > -0.3)]
    df_filtered_returns = df_returns[filtered_etfs]
    return df_filtered_returns

# 除外後の52週リターンを計算し、NaNを除去して最後の20行を表示
df_filtered_returns = round(calculate_filtered_moving_52_week_return().dropna(how="all").dropna(axis=1, thresh=80), 2)
display(df_filtered_returns.tail(20))


def plot_moving_52_week_return():
    """52週間の移動平均リターンをプロットする関数"""
    df_returns = df_filtered_returns
    df_long = pd.melt(df_returns.reset_index(), id_vars=['Date'], var_name='ETF', value_name='52 Week Return')

    # Plotly Expressを使用してインタラクティブなグラフを作成
    fig = px.line(df_long, x='Date', y='52 Week Return', color='ETF', 
                  title='ETFの52週間移動平均リターン', labels={'52 Week Return': '52週間リターン', 'Date': '日付'})

    # グラフを表示
    fig.show()

# 52週間の移動平均リターンをプロット
plot_moving_52_week_return()

FEPI: 1.82
SHOC: 1.76
CHPS: 1.74
FLOW: 1.40
MSFU: 1.13
SMH: 1.11
JGRO: 1.09
XLK: 1.05
SOXX: 1.00
IXN: 0.98
QQQ: 0.96
VGT: 0.96
DXJ: 0.92
TECL: 0.92
MGK: 0.88
QLD: 0.88
PSI: 0.87
VONG: 0.87
XNTK: 0.87
SOXL: 0.87
FTXL: 0.85
XSD: 0.85
RTH: 0.84
FTLS: 0.84
VUG: 0.84
GRID: 0.83
TQQQ: 0.83
OEF: 0.81
AIRR: 0.80
IOO: 0.79
AIQ: 0.79
QTEC: 0.78
NIKA: 0.78
MGC: 0.77
SPYG: 0.77
VOOG: 0.77
FXL: 0.75
DGRW: 0.74
SPLG: 0.73
VOO: 0.73
IVV: 0.73
SPY: 0.73
ESGU: 0.73
VV: 0.72
SUSA: 0.72
SPTM: 0.71
VONE: 0.71
CIBR: 0.71
FIW: 0.70
QQQE: 0.70
GLDM: 0.70
QQQM: 0.70
DXJS: 0.70
IAU: 0.69
VTHR: 0.69
GLD: 0.68
VTI: 0.68
SPXL: 0.68
UPRO: 0.68
EPI: 0.67
VIG: 0.67
SOXQ: 0.65
ARVR: 0.65
SPUU: 0.65
KRMA: 0.64
URA: 0.64
BUG: 0.64
XLV: 0.63
NXTG: 0.63
VCR: 0.63
FTCS: 0.62
XLC: 0.61
LOUP: 0.61
RDVY: 0.61
CURE: 0.60
TOK: 0.59
EMSF: 0.59
VHT: 0.59
SPYV: 0.59
VOOV: 0.59
IXJ: 0.58
DRIV: 0.58
SEMI: 0.56
VIS: 0.56
PPH: 0.55
VOT: 0.55
FXZ: 0.55
AMZU: 0.55
XLY: 0.55
ACWI: 0.55
DLN: 0.54
HACK: 0.54
RSP: 0.54
XLI: 0.54
MILN: 0.53

,XLK,DXJ,RTH,FTLS,GRID,OEF,AIRR,IOO,MGC,DGRW,...,RSP,XLI,IVOG,VT,DIA,MGV,VDC,XLB,HEDJ,EXI
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-16,0.32,0.34,0.03,0.09,0.11,0.17,0.14,0.15,0.15,0.10,...,0.03,0.11,0.06,0.11,0.07,0.04,-0.00,0.04,0.15,0.14
2023-10-23,0.24,0.32,0.00,0.06,0.02,0.10,0.05,0.10,0.08,0.02,...,-0.05,0.02,-0.02,0.05,-0.01,-0.04,-0.07,0.00,0.10,0.06
2023-10-30,0.42,0.38,0.09,0.11,0.06,0.22,0.12,0.20,0.19,0.10,...,0.03,0.07,0.06,0.13,0.05,0.01,-0.02,0.04,0.15,0.12
2023-11-06,0.35,0.34,0.04,0.10,-0.00,0.17,0.05,0.15,0.14,0.07,...,-0.04,0.03,-0.01,0.07,0.02,-0.02,-0.04,-0.05,0.10,0.05
2023-11-13,0.38,0.36,0.05,0.10,0.04,0.20,0.09,0.18,0.17,0.08,...,0.01,0.06,0.03,0.11,0.04,-0.00,-0.05,0.00,0.13,0.10
2023-11-20,0.38,0.32,0.05,0.10,0.03,0.19,0.08,0.17,0.17,0.07,...,-0.00,0.05,0.02,0.10,0.03,-0.01,-0.06,-0.02,0.12,0.08
2023-11-27,0.37,0.36,0.05,0.09,0.05,0.19,0.12,0.16,0.16,0.07,...,0.01,0.06,0.04,0.09,0.05,-0.01,-0.06,-0.00,0.13,0.09
2023-12-04,0.43,0.31,0.10,0.12,0.07,0.23,0.19,0.20,0.21,0.10,...,0.05,0.10,0.08,0.12,0.08,0.02,-0.05,0.01,0.17,0.12
2023-12-11,0.50,0.29,0.15,0.15,0.15,0.29,0.30,0.24,0.26,0.15,...,0.11,0.15,0.15,0.17,0.13,0.06,-0.01,0.08,0.21,0.18
